In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tempfile
from pytube import YouTube
import mediapipe as mp
import pickle

# Load your pose classification model
with open('../models/squat/squat.pkl', 'rb') as f:
    model = pickle.load(f)

# MediaPipe Pose setup
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)
mp_drawing = mp.solutions.drawing_utils

# Function to download YouTube video
def download_youtube_video(url: str):
    yt = YouTube(url)
    stream = yt.streams.filter(file_extension='mp4', progressive=True).get_highest_resolution()
    download_path = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4").name
    stream.download(filename=download_path)
    return download_path

# Function to extract pose keypoints
def extract_keypoints(results):
    if not results.pose_landmarks:
        return np.zeros(33 * 4)  # x, y, z, visibility for each landmark
    keypoints = []
    for lm in results.pose_landmarks.landmark:
        keypoints.extend([lm.x, lm.y, lm.z, lm.visibility])
    return np.array(keypoints)

# Label each frame and output CSV
def label_video(video_path, output_csv="downloads/output.csv"):
    cap = cv2.VideoCapture(video_path)
    frame_labels = []

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Resize for better performance if needed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        keypoints = extract_keypoints(results)

        if keypoints.sum() == 0:
            label = "NoPose"
        else:
            label = model.predict([keypoints])[0]

        frame_labels.append({
            "frame": frame_idx,
            "label": label
        })
        frame_idx += 1

    cap.release()

    df = pd.DataFrame(frame_labels)
    df.to_csv(output_csv, index=False)
    print(f"✅ Labels saved to {output_csv}")

# --- Run the script ---
if __name__ == "__main__":
    video_url = input("Enter YouTube URL: ")
    video_path = download_youtube_video(video_url)
    label_video(video_path)

I0000 00:00:1754127404.573613 9198710 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1754127404.649693 9198869 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754127404.660887 9198866 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


HTTPError: HTTP Error 400: Bad Request